In [5]:
import pandas as pd
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("data/cleaned_agridata.csv")
mydb = psycopg2.connect(
    host="localhost",
    database="Agriculture",      
    user="postgres",
    password="pdb143",
    port="5432"
)

conn = mydb.cursor()


In [9]:
query = """SELECT * FROM "Agri";
"""
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\3867683936.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Dist_Code,Year,State_Code,State_Name,Dist_Name,Rice_Area_1000_Ha,Rice_Production_1000_Tons,Rice_Yield_Kg_Per_Ha,Wheat_Area_1000_Ha,Wheat_Production_1000_Tons,...,Sugarcane_Yield_Kg_Per_Ha,Cotton_Area_1000_Ha,Cotton_Production_1000_Tons,Cotton_Yield_Kg_Per_Ha,Fruits_Area_1000_Ha,Vegetables_Area_1000_Ha,Fruits_And_Vegetables_Area_1000_Ha,Potatoes_Area_1000_Ha,Onion_Area_1000_Ha,Fodder_Area_1000_Ha
0,1,1966,14,Chhattisgarh,Durg,548.0,185.0,337.59,44.0,20.0,...,1777.78,0.0,0.0,0.0,5.95,6.64,12.59,0.01,0.60,0.47
1,1,1967,14,Chhattisgarh,Durg,547.0,409.0,747.71,50.0,26.0,...,1500.00,0.0,0.0,0.0,5.77,7.24,13.02,0.01,0.56,1.23
2,1,1968,14,Chhattisgarh,Durg,556.3,468.0,841.27,53.7,30.0,...,1000.00,0.0,0.0,0.0,5.41,7.40,12.81,0.10,0.58,1.02
3,1,1969,14,Chhattisgarh,Durg,563.4,400.8,711.40,49.4,26.5,...,1900.00,0.0,0.0,0.0,5.52,7.16,12.69,0.01,0.56,0.84
4,1,1970,14,Chhattisgarh,Durg,571.6,473.6,828.55,44.2,29.0,...,2000.00,0.0,0.0,0.0,5.45,7.19,12.64,0.02,0.52,0.42


In [ ]:
# 1.Year-wise Trend of Rice Production Across States (Top 3)

query = """
WITH Top3_States AS (
    SELECT "State_Name",
           SUM("Rice_Production_1000_Tons") AS total_rice_production
    FROM "Agri"
    GROUP BY "State_Name"
    ORDER BY total_rice_production DESC
    LIMIT 3
)
SELECT a."Year",
       a."State_Name",
       SUM(a."Rice_Production_1000_Tons") AS yearly_rice_production
FROM "Agri" a
JOIN Top3_States t
  ON a."State_Name" = t."State_Name"
GROUP BY a."Year", a."State_Name"
ORDER BY a."Year", yearly_rice_production DESC;
"""

data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\3556479467.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Year,State_Name,yearly_rice_production
0,1966,West Bengal,4819.40
1,1966,Uttar Pradesh,2473.50
2,1966,Punjab,338.00
3,1967,West Bengal,4865.77
4,1967,Uttar Pradesh,2969.00


In [ ]:
# 2.Top 5 Districts by Wheat Yield Increase Over the Last 5 Years

query = """
WITH Year_Range AS (
    -- Get the last 5 years available in the dataset
    SELECT MAX("Year") AS max_year,
           MAX("Year") - 4 AS min_year
    FROM "Agri"
),
District_Yield AS (
    -- Get average yield for each district in first and last year
    SELECT a."Dist_Name",
           a."State_Name",
           MIN(CASE WHEN a."Year" = y.min_year THEN a."Wheat_Yield_Kg_Per_Ha" END) AS yield_start,
           MIN(CASE WHEN a."Year" = y.max_year THEN a."Wheat_Yield_Kg_Per_Ha" END) AS yield_end
    FROM "Agri" a
    CROSS JOIN Year_Range y
    WHERE a."Year" BETWEEN y.min_year AND y.max_year
    GROUP BY a."Dist_Name", a."State_Name"
),
Yield_Change AS (
    SELECT "Dist_Name",
           "State_Name",
           (yield_end - yield_start) AS yield_increase
    FROM District_Yield
    WHERE yield_start IS NOT NULL AND yield_end IS NOT NULL
)
SELECT "Dist_Name", "State_Name", yield_increase
FROM Yield_Change
ORDER BY yield_increase DESC
LIMIT 5;
"""
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\2358110296.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Dist_Name,State_Name,yield_increase
0,Chamba,Himachal Pradesh,2015.44
1,Vidisha,Madhya Pradesh,1815.15
2,Dewas,Madhya Pradesh,1800.55
3,Damoh,Madhya Pradesh,1645.57
4,Darjeeling,West Bengal,1586.63


In [ ]:
# 3.States with the Highest Growth in Oilseed Production (5-Year Growth Rate)


query = """
WITH Year_Range AS (
    SELECT MAX("Year") AS max_year,
           MAX("Year") - 4 AS min_year
    FROM "Agri"
),
State_Oilseeds AS (
    SELECT a."State_Name",
           SUM(CASE WHEN a."Year" = y.min_year THEN a."Oilseeds_Production_1000_Tons" END) AS prod_start,
           SUM(CASE WHEN a."Year" = y.max_year THEN a."Oilseeds_Production_1000_Tons" END) AS prod_end
    FROM "Agri" a
    CROSS JOIN Year_Range y
    WHERE a."Year" BETWEEN y.min_year AND y.max_year
    GROUP BY a."State_Name"
),
Growth AS (
    SELECT "State_Name",
           prod_start,
           prod_end,
           CASE 
               WHEN prod_start = 0 OR prod_start IS NULL THEN NULL
               ELSE ROUND( ((prod_end - prod_start) / prod_start * 100)::numeric, 2 )
           END AS growth_rate_percent
    FROM State_Oilseeds
)
SELECT "State_Name", prod_start, prod_end, growth_rate_percent
FROM Growth
WHERE growth_rate_percent IS NOT NULL
ORDER BY growth_rate_percent DESC
LIMIT 5;
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\4234085902.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,State_Name,prod_start,prod_end,growth_rate_percent
0,Andhra Pradesh,2242.26,2876.52,28.29
1,Haryana,899.10,1134.70,26.20
2,Orissa,698.57,535.34,-23.37
3,Madhya Pradesh,7107.74,0.00,-100.00
4,Jharkhand,32.13,0.00,-100.00


In [ ]:
# 4.District-wise Correlation Between Area and Production for Major Crops (Rice, Wheat, and Maize)

query = """
SELECT "Dist_Name",
       "State_Name",
       corr("Rice_Area_1000_Ha", "Rice_Production_1000_Tons")   AS rice_corr,
       corr("Wheat_Area_1000_Ha", "Wheat_Production_1000_Tons") AS wheat_corr,
       corr("Maize_Area_1000_Ha", "Maize_Production_1000_Tons") AS maize_corr
FROM "Agri"
GROUP BY "Dist_Name", "State_Name"
ORDER BY "State_Name", "Dist_Name";
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\1288093954.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Dist_Name,State_Name,rice_corr,wheat_corr,maize_corr
0,Ananthapur,Andhra Pradesh,0.737403,0.792199,0.958003
1,Chittoor,Andhra Pradesh,0.793966,0.908901,0.949190
2,East Godavari,Andhra Pradesh,0.646491,NaN,0.957461
3,Guntur,Andhra Pradesh,0.433883,0.676052,0.983265
4,Kadapa YSR,Andhra Pradesh,0.419919,0.913140,0.953639


In [ ]:
# 5.Yearly Production Growth of Cotton in Top 5 Cotton Producing States

query = """
WITH yearly_production AS (
    SELECT 
        a."Year",
        a."State_Name",
        SUM(a."Cotton_Production_1000_Tons") AS yearly_cotton_production
    FROM "Agri" a
    WHERE a."State_Name" IN ('Gujarat', 'Maharashtra', 'Punjab', 'Haryana', 'Telangana')
    GROUP BY a."Year", a."State_Name"
),

last_five_years AS (
    SELECT DISTINCT "Year"
    FROM yearly_production
    ORDER BY "Year" DESC
    LIMIT 5
)

SELECT 
    y."State_Name",
    y."Year",
    y.yearly_cotton_production,
    ROUND(
        (
            (y.yearly_cotton_production - LAG(y.yearly_cotton_production) 
                OVER (PARTITION BY y."State_Name" ORDER BY y."Year")) 
            / NULLIF(LAG(y.yearly_cotton_production) 
                OVER (PARTITION BY y."State_Name" ORDER BY y."Year"), 0)
        )::numeric, 2
    ) AS yoy_growth_percent
FROM yearly_production y
JOIN last_five_years l ON y."Year" = l."Year"
ORDER BY y."State_Name", y."Year";
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\1597484145.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,State_Name,Year,yearly_cotton_production,yoy_growth_percent
0,Gujarat,2013,1892.07,NaN
1,Gujarat,2014,1662.04,-0.12
2,Gujarat,2015,1357.65,-0.18
3,Gujarat,2016,928.02,-0.32
4,Gujarat,2017,1820.28,0.96


In [ ]:
# 6.Districts with the Highest Groundnut Production in 2020
query = """
SELECT 
    "Dist_Name",
    "State_Name",
    SUM("Groundnut_Production_1000_Tons") AS total_groundnut_production
FROM "Agri"
WHERE "Year" = 2017
GROUP BY "Dist_Name", "State_Name"
ORDER BY total_groundnut_production DESC
LIMIT 10;
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\113673862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Dist_Name,State_Name,total_groundnut_production
0,Jamnagar,Gujarat,977.65
1,Junagadh,Gujarat,945.49
2,Rajkot,Gujarat,873.87
3,Bikaner,Rajasthan,514.09
4,Ananthapur,Andhra Pradesh,454.94


In [ ]:
# 7.Annual Average Maize Yield Across All States
query = """
SELECT 
    "Year","State_Name",
    ROUND(AVG("Maize_Yield_Kg_Per_Ha")::numeric, 2) AS avg_maize_yield
FROM "Agri"
GROUP BY "Year","State_Name"
ORDER BY "Year";
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\871470817.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Year,State_Name,avg_maize_yield
0,1966,Telangana,724.06
1,1966,Maharashtra,835.87
2,1966,Andhra Pradesh,822.73
3,1966,Jharkhand,1029.92
4,1966,Orissa,876.95


In [ ]:
# 8.Total Area Cultivated for Oilseeds in Each State
query = """
SELECT 
    "State_Name",
    SUM("Oilseeds_Area_1000_Ha") AS total_oilseeds_area
FROM "Agri"
GROUP BY "State_Name"
ORDER BY total_oilseeds_area DESC;
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\2653555820.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,State_Name,total_oilseeds_area
0,Madhya Pradesh,189945.92
1,Rajasthan,142433.47
2,Maharashtra,134069.16
3,Gujarat,130726.71
4,Karnataka,92527.23


In [ ]:
# 9.Districts with the Highest Rice Yield

query = """
SELECT 
    "Dist_Name",
    "State_Name",
    ROUND(AVG("Rice_Yield_Kg_Per_Ha")::numeric, 2) AS avg_rice_yield
FROM "Agri"
GROUP BY "Dist_Name", "State_Name"
ORDER BY avg_rice_yield DESC
LIMIT 10;
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\2434517946.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,Dist_Name,State_Name,avg_rice_yield
0,Ludhiana,Punjab,3650.43
1,Sangrur,Punjab,3532.90
2,Thirunelveli,Tamil Nadu,3387.51
3,Bhatinda,Punjab,3363.16
4,Madurai,Tamil Nadu,3276.98


In [ ]:
# 10.Compare the Production of Wheat and Rice for the Top 5 States Over 10 Years

query = """
WITH top_states AS (
    SELECT 
        "State_Name",
        SUM("Wheat_Production_1000_Tons" + "Rice_Production_1000_Tons") AS total_cereal_production
    FROM "Agri"
    GROUP BY "State_Name"
    ORDER BY total_cereal_production DESC
    LIMIT 5
),

last_10_years AS (
    SELECT DISTINCT "Year"
    FROM "Agri"
    ORDER BY "Year" DESC
    LIMIT 10
),

yearly_production AS (
    SELECT 
        a."Year",
        a."State_Name",
        SUM(a."Rice_Production_1000_Tons") AS rice_production,
        SUM(a."Wheat_Production_1000_Tons") AS wheat_production
    FROM "Agri" a
    JOIN top_states t ON a."State_Name" = t."State_Name"
    JOIN last_10_years y ON a."Year" = y."Year"
    GROUP BY a."Year", a."State_Name"
)

SELECT 
    "State_Name",
    "Year",
    rice_production,
    wheat_production
FROM yearly_production
ORDER BY "State_Name", "Year";
""" 
data = pd.read_sql_query(query, mydb)
display(data.head())

C:\Users\vigne\AppData\Local\Temp\ipykernel_30364\871990775.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, mydb)


,State_Name,Year,rice_production,wheat_production
0,Haryana,2008,3299.0,11360.0
1,Haryana,2009,3628.0,10488.0
2,Haryana,2010,3465.0,11578.0
3,Haryana,2011,3757.0,13119.0
4,Haryana,2012,3941.0,11117.0
